#### High-level "Notebook API"

In [1]:
import sys 
import os 
sys.path.insert(0, '..')


# from biosimulator_processes.api.instance import 

In [2]:
from biosimulator_processes.io import read_report_outputs
from biosimulator_processes import CORE
from biosimulator_processes.helpers import plot_utc_outputs, plot_ode_output_data

In [3]:
from biosimulator_processes.processes.amici_process import UtcAmici
from biosimulator_processes.processes.copasi_process import UtcCopasi
from biosimulator_processes.processes.tellurium_process import UtcTellurium

In [4]:
omex_dir = '../test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator'
omex_fp = omex_dir + '.omex'
sbml_fp = os.path.join(omex_dir, 'BIOMD0000000012_url.xml')
sedml_fp = os.path.join(omex_dir, 'simulation.sedml')
reports_path = os.path.join(omex_dir, 'reports.h5')
selected_species = 'cI mRNA'

In [5]:
utc_tellurium = UtcTellurium(config={'model': {'model_source': omex_dir + '.omex'}})
tellurium_results = utc_tellurium.update()

In [6]:
utc_copasi = UtcCopasi(config={'model': {'model_source': omex_dir + '.omex'}})
copasi_results = utc_copasi.update()

In [7]:
copasi_results['floating_species'][selected_species][:3], copasi_results['floating_species'][selected_species][-1]

In [8]:
utc_copasi.plot_results()

In [9]:
utc_amici = UtcAmici(config={'model': {'model_source': omex_dir + '.omex'}})
amici_results = utc_amici.update()

In [10]:
tellurium_results['floating_species'][selected_species][:600][-1], tellurium_results['floating_species'][selected_species].shape

In [11]:
amici_results['floating_species'][selected_species][:600][-1], amici_results['floating_species'][selected_species].shape

In [12]:
copasi_results['floating_species'][selected_species].shape

In [13]:
from biosimulator_processes.api.compare import generate_comparison
import numpy as np 
import seaborn as sns

outputs = [
    amici_results['floating_species'][selected_species][:600], 
    copasi_results['floating_species'][selected_species],
    tellurium_results['floating_species'][selected_species][:600],
]
comparison = generate_comparison(
    outputs=outputs,
    simulators=['amici', 'copasi', 'tellurium'],
    method='prox'
)

In [14]:
comparison.data

In [15]:
d = dict(zip(
    ['amici', 'copasi', 'tellurium'],
    outputs
))

import matplotlib.pyplot as plt 

plt.figure(figsize=(20, 8))
for sim, output in d.items():
    sns.lineplot(x=amici_results['time'][:600], y=output, label=sim)
plt.title(selected_species)

In [16]:
comparison.data

In [17]:
# TODO: run composite here

In [18]:
from process_bigraph import Composite


utc_comparison_doc = {
    'utc_tellurium': {
        '_type': 'step',
        'address': 'local:utc-tellurium',
        'config': {
            'model': {
                'model_source': omex_dir + '.omex'
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['floating_species_store'],
            'model_parameters': ['model_parameters_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['floating_species_store'],
        }
    }
}

In [19]:
wf = Composite(
    config={
        'state': utc_comparison_doc,
        'bridge': {
            'time': ['time_store'],
            'floating_species': ['floating_species_store']
        }
    },
    core=CORE
)

In [20]:
wf.run(1)

In [ ]:
utc_tellurium.inputs()

In [21]:
from biosimulator_processes.execute import exec_utc_comparison

In [22]:
comparison = exec_utc_comparison(omex_fp=omex_fp, simulators=['amici', 'copasi', 'tellurium'], comparison_id='repressilator-comparison')

In [23]:
comparison